In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing

In [11]:
Coats = pd.read_table("Dan Coats.txt", header =None)
Coats.columns = ['text']
Coats['author'] = 'Dan Coats'

Mattis = pd.read_table("James Mattis.txt", header =None)
Mattis.columns = ['text']
Mattis['author'] = 'James Mattis'

Kelly = pd.read_table("John Kelly.txt", header =None)
Kelly.columns = ['text']
Kelly['author'] = 'John Kelly'

Hassett = pd.read_table("Kevin Hassett.txt", header =None)
Hassett.columns = ['text']
Hassett['author'] = 'Kevin Hassett'

Nielsen = pd.read_table("Kirstjen Nielsen.txt", header =None)
Nielsen.columns = ['text']
Nielsen['author'] = 'Kirstjen Nielsen'

Kudlow = pd.read_table("Larry Kudlow.txt", header =None)
Kudlow.columns = ['text']
Kudlow['author'] = 'Larry Kudlow'

Pence = pd.read_table("Mike Pence.txt", header =None)
Pence.columns = ['text']
Pence['author'] = 'Mike Pence'

Pompeo = pd.read_table("Mike Pompeo.txt", header =None)
Pompeo.columns = ['text']
Pompeo['author'] = 'Mike Pompeo'

train = pd.DataFrame()
train = pd.concat([Coats, Mattis,Kelly,Hassett,Nielsen,Kudlow,Pence,Pompeo])

# Convert the author strings into numbers
#train['author'] = train['author'].apply(lambda x: possibleAuthors[x])
#print(train)


text = train['text'].tolist()
author = train['author'].tolist()

In [12]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(text)]

In [13]:
documents[:2]

[TaggedDocument(words='Good evening, and sincere thanks to the Atlantic Council, Le Figaro, and the Tocqueville Foundation for organizing this important conversation about Democracy in the West.\xa0', tags=[0]),
 TaggedDocument(words='I enjoyed catching up with Atlantic council board member and a long-time friend Ambassador Boyden Gray and the Atlantic Council’s Executive Vice President Damon Wilson just a few minutes ago.', tags=[1])]

In [16]:
N_DIMS = 100
N_EPOCHS = 50

d2v = Doc2Vec(seed=1,
            workers=multiprocessing.cpu_count(),
            vector_size=N_DIMS,
            dm=0,  # use distributed bag of words

            min_count=0,
            window=15,
            epochs=N_EPOCHS)

print("Building vocab...")
d2v.build_vocab(documents)
#print("Word2Vec vocabulary length:", len(d2v.wv.vocab))
#print("num shuffled visits: ", d2v.corpus_count)
print("Training...")
d2v.train(documents, total_examples=d2v.corpus_count, epochs=d2v.epochs)
print("Saving model...")
#d2v.save(w2v_file)

Building vocab...
Training...


In [20]:
vectors = [d2v.infer_vector(document.words) for document in documents]

In [24]:
### Test the model
X_train, X_test, y_train, y_test = train_test_split(vectors, author, test_size=0.25, random_state=1337)
svm = LinearSVC()
svm.fit(X_train, y_train)

predictions = svm.predict(X_test)
#print(list(predictions[0:10]))
 
from sklearn.metrics import accuracy_score
print("The Linear SVC model is accurate: ", round((accuracy_score(y_test, predictions) *100),2), "% of the time.")



The Linear SVC model is accurate:  72.64 % of the time.


In [26]:
opEd = pd.read_table("OpEd.txt", header =None)
opEd.columns = ['text']

test = opEd['text'].tolist()
test_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(test)]
X_test = [d2v.infer_vector(document.words) for document in test_documents]

# testVector = vectorizer.fit_transform(test)
# X_test=vectorizer.transform(test)
# print(vectors.shape)


predictions = svm.predict(X_test)

#predictedAuthor = mode(predictions)


predictedAuthordf = pd.DataFrame(predictions)
predictedAuthordf.columns = ['Author']
predictedAuthordf = predictedAuthordf['Author'].value_counts().reset_index()
predictedAuthordf = pd.DataFrame(predictedAuthordf)
predictedAuthordf.columns = ['Author','Count']
predictedAuthordf["Probability"] = predictedAuthordf["Count"]/(predictedAuthordf['Count'].sum())
predictedAuthordf["logLikelihood"] = np.log( predictedAuthordf["Probability"])

prediction = predictedAuthordf['logLikelihood'].idxmax()
predictedAuthor = predictedAuthordf.at[prediction,'Author']

print(predictedAuthordf,"\n\n")


#predictedAuthor = (list(possibleAuthors.keys())[list(possibleAuthors.values()).index(predictedAuthor)]) 
print("The predicted author is: ", predictedAuthor)

          Author  Count  Probability  logLikelihood
0     John Kelly     11     0.407407      -0.897942
1   Larry Kudlow      8     0.296296      -1.216395
2    Mike Pompeo      5     0.185185      -1.686399
3      Dan Coats      2     0.074074      -2.602690
4  Kevin Hassett      1     0.037037      -3.295837 


The predicted author is:  John Kelly
